## 1. Downloading Weather Data from ERA5

In [ ]:
import cdsapi

c = cdsapi.Client()

path_to_save = 'data/'

### API Request for PL variables
c.retrieve(
    'reanalysis-era5-pressure-levels',
    {
        'product_type': 'ensemble_members',
        'variable': [
            'fraction_of_cloud_cover', 'geopotential', 'potential_vorticity',
            'relative_humidity', 'specific_humidity', 'temperature',
            'u_component_of_wind', 'v_component_of_wind',
        ],
        'pressure_level': [
            '50', '70', '100',
            '125', '150', '175',
            '200', '225', '250',
            '300', '350', '400',
            '450', '500', '550',
            '600', '700', '800',
            '900', '1000',
        ],
        'year': '2018',
        'month': '12',
        'day': '20',
        'time': [
            '00:00', '03:00', '06:00',
            '09:00', '12:00', '15:00',
            '18:00', '21:00',
        ],
        'area': [
            70, -20, 20,
            40,
        ],
        'format': 'netcdf',
    },
    path_to_save + 'PL_20-12-2018_EU.nc')

### API Request for SF variables
c.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type': [
            'ensemble_members',
        ],
        'variable': [
            'surface_solar_radiation_downwards', 'top_net_thermal_radiation',
        ],
        'year': '2018',
        'month': '12',
        'day': '20',
        'time': [
            '00:00', '03:00', '06:00',
            '09:00', '12:00', '15:00',
            '18:00', '21:00',
        ],
        'area': [
            70, -20, 20,
            40,
        ],
        'format': 'netcdf',
    },
    path_to_save + 'SF_20-12-2018_EU.nc')

## 2. Running CLIMaCCF library on the downloaded data

## 3. Building interpolation over the output of CLIMaCCF for the ROC using CASADI

In [ ]:
from roc3.weather import *
from importlib import reload
from roc3.rfp import *
from roc3.apm import *
from roc3.bada4 import *
import pickle
import xarray as xr
import numpy as np


# output of climaccf should be inputted here as ds with BFFM2_Coef = True
def weather_intrp_save_pickle (ds, year, month, day, path_save):
    
    n_la = len(ds['latitude'].values)
    n_lo = len(ds['longitude'].values)
    n_t =  len(ds['time'].values)
    n_l =  len(ds['level'].values)
    n_m =  len(ds['number'].values)

    T = np.zeros((n_t,n_l,1,n_la,n_lo))
    U = np.zeros((n_t,n_l,1,n_la,n_lo))
    V = np.zeros((n_t,n_l,1,n_la,n_lo))

    aCCF_H2O = np.zeros((n_t,n_l,1,n_la,n_lo))
    
    r1 = np.zeros((n_t,n_l,1,n_la,n_lo))
    r2 = np.zeros((n_t,n_l,1,n_la,n_lo))
    r3 = np.zeros((n_t,n_l,1,n_la,n_lo))
    r4 = np.zeros((n_t,n_l,1,n_la,n_lo))
    r5 = np.zeros((n_t,n_l,1,n_la,n_lo))
    r6 = np.zeros((n_t,n_l,1,n_la,n_lo))
    r7 = np.zeros((n_t,n_l,1,n_la,n_lo))
    r8 = np.zeros((n_t,n_l,1,n_la,n_lo))
    r9 = np.zeros((n_t,n_l,1,n_la,n_lo))
    r10 = np.zeros((n_t,n_l,1,n_la,n_lo))

    aCCF_NOx = np.zeros((n_t,n_l,1,n_la,n_lo))
    olr = np.zeros((n_t,n_l,1,n_la,n_lo))

    for i in range(n_t):
        for j in range(n_l):
            T [i,j,:,:,:]  = np.mean(ds['t'].values[i,:, j,:,:],axis=0)
            U [i,j,:,:,:]  = np.mean(ds['u'].values[i,:, j,:,:],axis=0)
            V [i,j,:,:,:]  = np.mean(ds['v'].values[i,:, j,:,:],axis=0)
            aCCF_H2O [i,j,:,:,:] = np.mean(ds['aCCF_H2O'].values[i,:, j,:,:],axis=0)
            aCCF_NOx [i,j,:,:,:] = np.mean(ds['aCCF_O3'].values[i,:, j,:,:],axis=0) + np.mean(ds['aCCF_CH4'].values[i,:, j,:,:],axis=0)
            r1 [i,j,:,:,:]  = ds['r'].values[i,0, j,:,:] 
            r2 [i,j,:,:,:]  = ds['r'].values[i,1, j,:,:] 
            r3 [i,j,:,:,:]  = ds['r'].values[i,2, j,:,:] 
            r4 [i,j,:,:,:]  = ds['r'].values[i,3, j,:,:] 
            r5 [i,j,:,:,:]  = ds['r'].values[i,4, j,:,:] 
            r6 [i,j,:,:,:]  = ds['r'].values[i,5, j,:,:] 
            r7 [i,j,:,:,:]  = ds['r'].values[i,6, j,:,:] 
            r8 [i,j,:,:,:]  = ds['r'].values[i,7, j,:,:] 
            r9 [i,j,:,:,:]  = ds['r'].values[i,8, j,:,:] 
            r10 [i,j,:,:,:]  = ds['r'].values[i,9, j,:,:] 
            olr [i,j,:,:,:] = np.mean(ds['olr'].values[i,:, j,:,:],axis=0)

    times = []

    for ii in range (n_t):
        timE = 3*ii
        times.append(1526190000.0 + timE * 3600)


    ds1 = xr.Dataset(
            {
                "T": (("times", "levels","member", "lats", "longs"), T),
                "U": (("times", "levels","member", "lats", "longs"), U),
                "V": (("times", "levels","member", "lats", "longs"), V),

                "r1": (("times", "levels","member", "lats", "longs"), r1/100),
                "r2": (("times", "levels","member", "lats", "longs"), r2/100),
                "r3": (("times", "levels","member", "lats", "longs"), r3/100),
                "r4": (("times", "levels","member", "lats", "longs"), r4/100),
                "r5": (("times", "levels","member", "lats", "longs"), r5/100),
                "r6": (("times", "levels","member", "lats", "longs"), r6/100),
                "r7": (("times", "levels","member", "lats", "longs"), r7/100),
                "r8": (("times", "levels","member", "lats", "longs"), r8/100),
                "r9": (("times", "levels","member", "lats", "longs"), r9/100),
                "r10": (("times", "levels","member", "lats", "longs"), r10/100),
                
                "aCCF_H2O": (("times", "levels","member", "lats", "longs"), aCCF_H2O),
                "aCCF_NOx": (("times", "levels","member", "lats", "longs"), aCCF_NOx),
                "olr": (("times", "levels","member", "lats", "longs"), olr),

            },
            {"times": times,
            "member": [0],
            "levels":  ds['level'].values,
            "lats": ds['latitude'].values,
            "longs" : ds['longitude'].values},
            )
    # ds1.to_netcdf(path_save + "main_roc_processed.nc")
    ds2 = ds1.isel(times = slice(0,4))
    for time_selected in ['00', '12']:
        for name_vars in list(ds1.data_vars.keys()):
            for time_index in range (4):
                if time_selected == '00':
                    index_time = 0
                else:
                    index_time = 4    
                ds2[name_vars].values [time_index, :, :, :, :] = ds1[name_vars].values [index_time, :, :, :, :]
        ds2.to_netcdf(path_save + f"main_roc_processed_{year}-{month}-{day}-T{time_selected}UTC.nc")

        ws = WeatherStore_4D(path_save + f"main_roc_processed_{year}-{month}-{day}-T{time_selected}UTC.nc", ll_resolution = 1)
        print(ws.values['U'].shape)
        ws.reduce_domain({'lat': (30, 70), 'lon': (-15, 40)})
        # ws.reduce_domain({'lat': (45, 55), 'lon': (-125, -100)}
        print(ws.values['U'].shape) 

        wm = ws.get_weather_model(1)

        with open(path_save + f"main_roc_processed_{year}-{month}-{day}-T{time_selected}UTC.pickle", 'wb') as handle:
            pickle.dump(wm, handle, protocol=pickle.HIGHEST_PROTOCOL)
    return wm 

month = 'Dec'
month_alph = '12'

year = '2018'

days =['20']#', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30']
 
for day in days: 
    ds = xr.open_dataset(f'data/env_processed_{year}_{month_alph}_{day}.nc', engine = 'h5netcdf')
    weather_intrp_save_pickle (ds, year, month, day, 'data/')

(4, 20, 1, 56, 131)
(4, 20, 1, 41, 56)
(4, 20, 1, 56, 131)
(4, 20, 1, 41, 56)
